In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 4"

In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
import optuna
import gc
xgb.__version__

/home/tunguz/.local/lib/python3.12/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


'2.1.2'

In [3]:
!nvidia-smi

Fri Jan 10 09:03:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro RTX 5000                Off |   00000000:01:00.0  On |                  N/A |
| N/A   49C    P8              6W /  110W |     397MiB /  16384MiB |     17%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
!nvidia-smi -L

GPU 0: Quadro RTX 5000 (UUID: GPU-0dad801b-446c-e905-a909-c5a1dc218d05)


In [5]:
train_x = pd.read_csv('../input/higgs_small_roc/xgtrain_ss.csv')
test_x = pd.read_csv('../input/higgs_small_roc/xgval_ss.csv')

In [6]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [7]:
dtrain = xgb.DMatrix(train_x, train_y)

In [8]:
dtest = xgb.DMatrix(test_x, test_y)


In [9]:
num_round = 1000

In [10]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']),
        'tree_method': trial.suggest_categorical('tree_method',['hist']),  # 'gpu_hist','hist'
        'device': trial.suggest_categorical('device',['cuda']),

        'lambda': trial.suggest_float('lambda',1e-3,10.0, log=True),
        'alpha': trial.suggest_float('alpha',1e-3,10.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.001,0.1, log=True),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    output = xgb.train(params, dtrain, num_round)
    
    #booster = output['booster']  # booster is the trained model
    #booster.set_param({'predictor': 'gpu_predictor'})

    predictions = output.predict(dtest)
    
    #predictions = predictions.compute()

    roc = roc_auc_score(test_y, predictions)
    
    return roc

In [11]:
study = optuna.create_study(direction='maximize')

[I 2025-01-10 09:03:09,672] A new study created in memory with name: no-name-da8fb20c-9cf5-4ca3-b49b-e80e0ebd5579


In [12]:
%%time
study.optimize(objective, n_trials=2)

[I 2025-01-10 09:03:11,320] Trial 0 finished with value: 0.7699487033371424 and parameters: {'objective': 'binary:logistic', 'tree_method': 'hist', 'device': 'cuda', 'lambda': 0.0032446799901623903, 'alpha': 1.2643835824095597, 'colsample_bytree': 0.8002904543125735, 'subsample': 0.737954475041446, 'learning_rate': 0.005715855155598703, 'max_depth': 3, 'min_child_weight': 79, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.7699487033371424.
[I 2025-01-10 09:03:12,472] Trial 1 finished with value: 0.7465490578890817 and parameters: {'objective': 'binary:logistic', 'tree_method': 'hist', 'device': 'cuda', 'lambda': 0.0012826615839869498, 'alpha': 0.05048725119491934, 'colsample_bytree': 0.940229958683032, 'subsample': 0.8600259499683084, 'learning_rate': 0.0014583269117000388, 'max_depth': 5, 'min_child_weight': 184, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.7699487033371424.


CPU times: user 2.73 s, sys: 187 ms, total: 2.92 s
Wall time: 2.28 s


In [13]:
%%time
study.optimize(objective, n_trials=1000)

[I 2025-01-10 09:03:15,565] Trial 2 finished with value: 0.7869756238017318 and parameters: {'objective': 'binary:logistic', 'tree_method': 'hist', 'device': 'cuda', 'lambda': 2.1805462149159607, 'alpha': 0.0013836111863006842, 'colsample_bytree': 0.7561514624457841, 'subsample': 0.8709006984791814, 'learning_rate': 0.01812245118310185, 'max_depth': 9, 'min_child_weight': 6, 'eval_metric': 'logloss'}. Best is trial 2 with value: 0.7869756238017318.
[I 2025-01-10 09:03:16,780] Trial 3 finished with value: 0.7748879710178369 and parameters: {'objective': 'binary:logistic', 'tree_method': 'hist', 'device': 'cuda', 'lambda': 1.0842300432513843, 'alpha': 0.0023413298737996295, 'colsample_bytree': 0.5797300394275516, 'subsample': 0.9421751462014964, 'learning_rate': 0.029313018930005205, 'max_depth': 11, 'min_child_weight': 213, 'eval_metric': 'logloss'}. Best is trial 2 with value: 0.7869756238017318.
[I 2025-01-10 09:03:18,248] Trial 4 finished with value: 0.7793769557291641 and parameters

CPU times: user 1h 41min 22s, sys: 9.72 s, total: 1h 41min 32s
Wall time: 1h 35min 25s


In [16]:
study.best_trial.params

{'objective': 'binary:logistic',
 'tree_method': 'hist',
 'device': 'cuda',
 'lambda': 4.081123667487544,
 'alpha': 0.018948321322930495,
 'colsample_bytree': 0.9959299202093657,
 'subsample': 0.7749822047878192,
 'learning_rate': 0.00945751906853162,
 'max_depth': 7,
 'min_child_weight': 1,
 'eval_metric': 'logloss'}

In [17]:
real_test_x = pd.read_csv('../input/higgs_small_roc/xgtest_ss.csv')

In [18]:
real_test_y = real_test_x['target']
real_test_x = real_test_x[real_test_x.columns.difference(['target'])]

In [19]:
drealtest = xgb.DMatrix(real_test_x, real_test_y)

In [20]:
params = study.best_trial.params

output = xgb.train(params, dtrain, num_round)
    


predictions = output.predict(drealtest)
    
roc_auc_score(real_test_y, predictions)

0.7841266979088475

In [21]:
%%time
params = study.best_trial.params
params['seed'] = 2022

output = xgb.train(params, dtrain, num_round)

predictions = output.predict(drealtest)
    
roc_auc_score(real_test_y, predictions)

CPU times: user 3.11 s, sys: 12 ms, total: 3.12 s
Wall time: 2.75 s


0.783972180239104

In [21]:
%%time 
scores = []

for jj in range(15):
    print(jj)
    params = study.best_trial.params
    params['seed'] = 5*jj**3 + 137

    output = xgb.dask.train(client, params, dtrain, num_round)

    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, drealtest)

    predictions = predictions.compute()

    scores.append(roc_auc_score(real_test_y, predictions))

0


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


1


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


2


...

13


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


14


/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)


CPU times: user 23.5 s, sys: 4.35 s, total: 27.9 s
Wall time: 5min 9s


In [22]:
scores

[0.8067397221533028,
 0.8066990070642219,
 0.8063826106965943,
 0.8064383115265237,
 0.8064385880739696,
 0.8062720699863317,
 0.8067426754713112,
 0.8064104376311154,
 0.8061099140433141,
 0.8061604909186776,
 0.8066853310103347,
 0.8062528838550297,
 0.8066905749760565,
 0.8061511717915344,
 0.8065482678346361]

In [24]:
import numpy as np
np.mean(scores)

0.8064481371355302